In [10]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms,datasets
import numpy as np
from pathlib import Path
from PIL import Image
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import sys 
sys.path.append('../Utils')
import configs

In [22]:
# ---------------- CONFIG ----------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_PRETRAINED = True      # use pretrained CNN
FREEZE_CNN = True          # freeze CNN backbone
BACKBONE = "resnet18"      # choose resnet18 or resnet50
NUM_CLASSES = 7
NUM_EPOCHS = 50
BATCH_SIZE = 16
LR = 1e-4
NUM_VIEWS = configs.NUM_VIEWS if hasattr(configs, "NUM_VIEWS") else 5

# Data paths from configs.py
MULTIVIEW_TRAIN_DIR, MULTIVIEW_TEST_DIR = configs.MULTIVIEW_TRAIN_DIR, configs.MULTIVIEW_TEST_DIR
TRAIN_FEATURES_DIR, TEST_FEATURES_DIR   = configs.TRAIN_FEATURES_DIR, configs.TEST_FEATURES_DIR

In [23]:
# ---------------- DATA ----------------
train_lbp = np.load(TRAIN_FEATURES_DIR / "features.npy")
train_labels = np.load(TRAIN_FEATURES_DIR / "labels.npy")
test_lbp  = np.load(TEST_FEATURES_DIR / "features.npy")
test_labels  = np.load(TEST_FEATURES_DIR / "labels.npy")
LBP_DIM = train_lbp.shape[1]
print(f"Train Shape  Features: {train_lbp.shape} | Labels: {train_labels.shape}")
print(f"Test Shape Features: {test_lbp.shape} | Labels: {test_labels.shape}")


Train Shape  Features: (2785, 26) | Labels: (2785,)
Test Shape Features: (670, 26) | Labels: (670,)


In [24]:
# Define Transformations 
tfm = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),
    "test": transforms.Compose([
        transforms.Resize(256), transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),
}

In [25]:
# Load Data From Multiview 
train_set = datasets.ImageFolder(root=MULTIVIEW_TRAIN_DIR, transform=tfm['train'])
test_set = datasets.ImageFolder(root=MULTIVIEW_TEST_DIR, transform=tfm['test'])
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Print dataset and loader information
print("\nDataset Info:")
print(f"Train size: {len(train_set)} | Test size: {len(test_set)} | Classes: {train_set.classes}")


Dataset Info:
Train size: 2785 | Test size: 670 | Classes: ['Ash', 'Beech', 'Douglas Fir', 'Oak', 'Pine', 'Red Oak', 'Spruce']


In [26]:
# ---------------- MODEL ----------------
def make_backbone(name="resnet18", pretrained=True):
    if name=="resnet18":
        m = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1 if pretrained else None)
        return nn.Sequential(*list(m.children())[:-1]), 512
    if name=="resnet50":
        m = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1 if pretrained else None)
        return nn.Sequential(*list(m.children())[:-1]), 2048

class FusionModel(nn.Module):
    def __init__(self, cnn_dim, lbp_dim, num_classes):
        super().__init__()
        self.cnn, self.cnn_dim = make_backbone(BACKBONE, USE_PRETRAINED)
        if FREEZE_CNN:
            for p in self.cnn.parameters(): p.requires_grad=False
        self.fc = nn.Sequential(
            nn.Linear(cnn_dim+lbp_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    def forward(self, x, lbp):
        f = self.cnn(x).view(x.size(0), -1)
        fused = torch.cat([f, lbp.to(f.device)],1)
        return self.fc(fused)

cnn_dim = 512 if BACKBONE=="resnet18" else 2048
model = FusionModel(cnn_dim, LBP_DIM, NUM_CLASSES).to(DEVICE)
crit, opt = nn.CrossEntropyLoss(), optim.Adam(filter(lambda p:p.requires_grad, model.parameters()), lr=LR)

In [27]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# ---------------- TRAIN ----------------
def train_model(model, train_loader, lbp_features, optimizer, criterion, device, num_epochs=10, metrics=None):
    """
    Train the fusion model.

    Args:
        model: torch.nn.Module
        train_loader: DataLoader
        lbp_features: np.ndarray
        optimizer: torch optimizer
        criterion: loss function
        device: cuda/cpu
        num_epochs: int
        metrics: dict of custom metrics { "metric_name": func(y_true, y_pred) }
    Returns:
        history: dict containing loss, acc, and metrics per epoch
    """
    history = {"loss": [], "acc": []}
    if metrics:
        for k in metrics.keys():
            history[k] = []

    for epoch in range(num_epochs):
        model.train()
        tot, correct, loss_sum = 0, 0, 0
        all_preds, all_labels = [], []

        for i, (x, y) in enumerate(train_loader):
            lbp = torch.tensor(lbp_features[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]).float()
            x, y, lbp = x.to(device), y.to(device), lbp.to(device)

            optimizer.zero_grad()
            out = model(x, lbp)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

            loss_sum += loss.item() * y.size(0)
            preds = out.argmax(1)
            correct += (preds == y).sum().item()
            tot += y.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

        epoch_loss = loss_sum / tot
        epoch_acc = 100 * correct / tot
        history["loss"].append(epoch_loss)
        history["acc"].append(epoch_acc)

        log = f"Epoch {epoch+1}/{num_epochs} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2f}%"
        if metrics:
            for name, fn in metrics.items():
                val = fn(all_labels, all_preds)
                history[name].append(val)
                log += f" | {name}: {val:.4f}"
        print(log)

    return history


# ---------------- TEST ----------------
def test_model(model, test_loader, lbp_features, device, metrics=None):
    """
    Evaluate the fusion model.

    Args:
        model: torch.nn.Module
        test_loader: DataLoader
        lbp_features: np.ndarray
        device: cuda/cpu
        metrics: dict of custom metrics { "metric_name": func(y_true, y_pred) }
    Returns:
        results: dict with accuracy + custom metrics
    """
    model.eval()
    preds, gts = [], []
    with torch.no_grad():
        for i, (x, y) in enumerate(test_loader):
            lbp = torch.tensor(lbp_features[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]).float()
            out = model(x.to(device), lbp.to(device))
            preds += out.argmax(1).cpu().tolist()
            gts += y.tolist()

    acc = accuracy_score(gts, preds) * 100
    results = {"acc": acc}
    if metrics:
        for name, fn in metrics.items():
            results[name] = fn(gts, preds)

    print("\nTest Results:")
    print(f"Accuracy: {acc:.2f}%")
    if metrics:
        for k, v in results.items():
            if k != "acc":
                print(f"{k}: {v:.4f}")
    print(classification_report(gts, preds, digits=4))

    return results


In [ ]:
# custom_metrics = {
#     "f1_macro": lambda y_true, y_pred: f1_score(y_true, y_pred, average="macro"),
#     "precision_macro": lambda y_true, y_pred: precision_score(y_true, y_pred, average="macro"),
#     "recall_macro": lambda y_true, y_pred: recall_score(y_true, y_pred, average="macro"),
# }

print("\nTraining Fusion Model...")
history = train_model(model, train_loader, train_lbp, opt, crit, DEVICE,
                      num_epochs=NUM_EPOCHS)

print("\nTesting Fusion Model...")
results = test_model(model, test_loader, test_lbp, DEVICE)



Training Fusion Model...
Epoch 1/50 | Loss: 1.6840 | Acc: 32.60% | f1_macro: 0.1752 | precision_macro: 0.1808 | recall_macro: 0.1948
Epoch 1/50 | Loss: 1.6840 | Acc: 32.60% | f1_macro: 0.1752 | precision_macro: 0.1808 | recall_macro: 0.1948


c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 2/50 | Loss: 1.5521 | Acc: 39.57% | f1_macro: 0.2316 | precision_macro: 0.3021 | recall_macro: 0.2472


c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 3/50 | Loss: 1.4622 | Acc: 44.81% | f1_macro: 0.2653 | precision_macro: 0.2626 | recall_macro: 0.2824


...

Epoch 9/50 | Loss: 1.2909 | Acc: 51.45% | f1_macro: 0.3439 | precision_macro: 0.4755 | recall_macro: 0.3498


c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 10/50 | Loss: 1.2666 | Acc: 51.63% | f1_macro: 0.3473 | precision_macro: 0.4253 | recall_macro: 0.3527


c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 11/50 | Loss: 1.2419 | Acc: 53.03% | f1_macro: 0.3836 | precision_macro: 0.5059 | recall_macro: 0.3774
Epoch 12/50 | Loss: 1.2315 | Acc: 53.18% | f1_macro: 0.3818 | precision_macro: 0.6017 | recall_macro: 0.3785
Epoch 12/50 | Loss: 1.2315 | Acc: 53.18% | f1_macro: 0.3818 | precision_macro: 0.6017 | recall_macro: 0.3785
Epoch 13/50 | Loss: 1.2186 | Acc: 53.82% | f1_macro: 0.3875 | precision_macro: 0.5382 | recall_macro: 0.3823
Epoch 13/50 | Loss: 1.2186 | Acc: 53.82% | f1_macro: 0.3875 | precision_macro: 0.5382 | recall_macro: 0.3823
Epoch 14/50 | Loss: 1.2108 | Acc: 54.18% | f1_macro: 0.3902 | precision_macro: 0.6120 | recall_macro: 0.3865
Epoch 14/50 | Loss: 1.2108 | Acc: 54.18% | f1_macro: 0.3902 | precision_macro: 0.6120 | recall_macro: 0.3865
Epoch 15/50 | Loss: 1.2113 | Acc: 54.22% | f1_macro: 0.4059 | precision_macro: 0.5123 | recall_macro: 0.3979
Epoch 15/50 | Loss: 1.2113 | Acc: 54.22% | f1_macro: 0.4059 | precision_macro: 0.5123 | recall_macro: 0.3979
Epoch 16/50 | Loss: